In [ ]:
#率変化(MGG(ルーレット))
import random
import os
import math
import csv 
import json
import numpy as np
import pandas as pd
type(np.int32(np.atleast_1d(np.int32(0))))
from operator import attrgetter


csv_name_list=os.listdir('C:\\Users\\onlyy\\Desktop\\jupyter notebook\\JWA調査\\datas')
#os.chdir('C:\\Users\\onlyy\\Desktop\\jupyter notebook\\JWA調査\\datas')
wavedate_sample=pd.read_csv(csv_name_list[1], encoding="shift_jis")
f_list=[]
for i in range(25):
    f_list.append(wavedate_sample.iloc[i][0])
    
def main():
    TK=open("JWA2.txt","w",encoding="UTF-8")
    TK.write("Hv1,s1,θ1,λ1,ωm1,Hv2,s2,θ2,λ2,ωm2")
    TK.write('\n')

    n_gene   = 64   # The number of genes.
    n_ind    = 100  # 偶数に設定he number of individuals in a population.
    #CXPB     = 0.95   # The probability of crossover.
    MUTPB    = 1.0   # The probability of individdual mutation.
    #MUTINDPB = 0.0080  # The probability of gene mutation.
    NGEN     = 20000   # The number of generation loop.
    n_parents=2     #2に設定
    CXPB     =0.85
    MUTINDPB = 0.0050
    CXPB_MAX =0.900
    CXPB_MIN =0.200
    M_MAX    =10.0/62.0
    M_MIN    =1.0/62.0/2.0
    f        =25.0
    for k in range(63):#63
        wavedate= pd.read_csv(csv_name_list[k], encoding="shift_jis")
        print(k+1)
        TK.write(str(csv_name_list[k])+":")
        random.seed()
        # --- Step1 : 母集団作成.
        pop = create_pop(n_ind, n_gene)
        set_fitness(get_parameter_from_ind,pop,wavedate)
        fit_sum = keisan(n_ind,pop)
        best_ind = min(pop, key=attrgetter("fitness"))   

        # --- 世代分の繰り返し.
        #print("Generation loop start.")
        #print("Generation: 0. Best fitness: " + str(best_ind.fitness))
        
        for g in range(NGEN):     
            # --- Step2 : 選択.
            #elitism = []
            #elitism.append(min(pop, key=attrgetter("fitness")))
            #elite=elitism[:]
            parents,pop = randomselection(n_ind,pop,n_parents)
            fit_sum=fit_sum-parents[0].fitness-parents[1].fitness
            #set_fitness(evalOneMax, parents)
            children = parents
            #worst_ind = max(pop, key=attrgetter("fitness"))
            #C_max = worst_ind.fitness
            #offspring,fitness_mean,fit_var = roulette(pop, n_ind,C_max)
            #print("offspring1"+str(len(offspring)))
            #offspring = tournament(pop, n_ind,tournsize)
  
            # --- Step3 : 交叉.
            crossover = []
            for child1, child2 in zip(children[::2], children[1::2]):
                if random.random() < CXPB:
                    child1, child2 = cxTwoPointCopy(child1, child2)
                    child1.fitness = None
                    child2.fitness = None
                crossover.append(child1)
                crossover.append(child2)
            #crossover.append(min(elite, key=attrgetter("fitness")))
            children = crossover[:]
            #print("offspring2"+str(len(offspring)))

            # --- Step4 : 突然変異.
            mutant = []
            for mut in (children):
                if random.random() < MUTPB:
                    mut = mutFlipBit(mut, indpb=MUTINDPB)
                    mut.fitness = None
                mutant.append(mut)
            #mutants=mutant[:]
            #print("mutants"+str(len(mutants)))
            #mutant.append(min(elite, key=attrgetter("fitness")))
            children = mutant[:]
            set_fitness(get_parameter_from_ind, children,wavedate)
            #print("offspring3"+str(len(offspring)))
        
             #-----Step5 : 生存選択
            family = children+parents
            offspring = []
            #for i in range(int(n_parents/2)):
                #worst_ind = max(family, key=attrgetter("fitness"))
                #C_max     = worst_ind.fitness
                #offspring = roulette(family,C_max)
                #offspring.append(random.choice(family))
            #family_sorted = sorted(family, key=attrgetter("fitness"))
            #offspring = offspring+family_sorted[:int(n_parents/2)]
        
            family_sorted = sorted(family, key=attrgetter("fitness"))
            for i in range(int(n_parents/2)):
                worst_ind = max(family_sorted[int(n_parents/2):], key=attrgetter("fitness"))
                C_max     = worst_ind.fitness
                offspring = roulette(family_sorted[int(n_parents/2):],C_max)
                #offspring.append(random.choice(family))
            offspring = offspring+family_sorted[:int(n_parents/2)]

        
            # --- 次世代を現世代に更新.
            pop = pop + offspring
            fit_sum = fit_sum+offspring[0].fitness+offspring[1].fitness
            fitness_mean =fit_sum/n_gene #,fit_var
            # --- Print best fitness in the population.
            best_ind = min(pop, key=attrgetter("fitness"))
            f_max = best_ind.fitness
            CXPB,MUTINDPB = adjustrating(CXPB,MUTINDPB,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean)
            #print("Generation: " + str(g+1) + ". Best fitness: " + str(best_ind.fitness)+ " mean: "+str(int(fitness_mean))+" MU: "+str(MUTINDPB))#+" CXPB: "+str(CXPB))  #+" variance: " + str(int(fit_var))
            #if best_ind.fitness <= 0.0 :
                #break
        lambda_1,s_1,theta_1,omega_1,Hv_1,lambda_2,s_2,theta_2,omega_2,Hv_2=get_parameter_from_ind(best_ind)   
        #print("Generation loop ended. The best individual: ")
        #print(best_ind)
        #print(str('%03.6f' %(Hv_1))+","+str(s_1)+","+str('%03.6f' %(theta_1))+","+str(lambda_1)+","+str('%03.6f' %(omega_1))+",")
        TK.write(str('%03.6f' %(Hv_1))+","+str(s_1)+","+str('%03.6f' %(theta_1))+","+str(lambda_1)+","+str('%03.6f' %(omega_1))+",")
        TK.write(str('%03.6f' %(Hv_2))+","+str(s_2)+","+str('%03.6f' %(theta_2))+","+str(lambda_2)+","+str('%03.6f' %(omega_2)))
        TK.write('\n')
    print("end")
    TK.close()
   
class Individual(np.ndarray):
    """Container of a individual."""
    fitness = None
    
    def __new__(cls, a):
        return np.asarray(a).view(cls)

def create_ind(n_gene):
    """Create a individual."""
    return Individual([random.randint(0, 1) for i in range(n_gene)])

def create_pop(n_ind, n_gene):
    """Create a population."""
    pop = []
    for i in range(n_ind):
        ind = create_ind(n_gene)
        pop.append(ind)
    return pop

def set_fitness(parameter,pop,wavedate):
    """Set fitnesses of each individual in a population."""
    for i, fit in zip(range(len(pop)), map(parameter, pop)):
        E=evalOneMax(fit,wavedate)
        pop[i].fitness = E

        
def evalOneMax(fit,wavedate):
    """Objective function."""
    errorfunc=0.0
    lambda_1, s_max1, theta_1, omega_1,Hv_1, lambda_2, s_max2, theta_2, omega_2,Hv_2 = fit
    #lambda_1 = (1.0*ind[0]+2*ind[10]+4*ind[20]+8*ind[30]+16*ind[40])*0.10+0.10
    #theta_1  = (1.0*ind[1]+2*ind[11]+4*ind[21]+8*ind[31]+16*ind[41]+32*ind[50]+64*ind[56])*2.8125
    #s_1      = (1.0*ind[2]+2*ind[12]+4*ind[22]+8*ind[32]+16*ind[42])+1.0
    #omega_1  = (1.0*ind[3]+2*ind[13]+4*ind[23]+8*ind[33]+16*ind[43]+32*ind[51])*0.040+0.20
    #Hv_1     = (1.0*ind[4]+2*ind[14]+4*ind[24]+8*ind[34]+16*ind[44]+32.0*ind[52]+64.0*ind[57]+128.0*ind[60])*0.050
    #lambda_2 = (1.0*ind[5]+2*ind[15]+4*ind[25]+8*ind[35]+16*ind[45])*0.10+0.10
    #theta_2  = (1.0*ind[6]+2*ind[16]+4*ind[26]+8*ind[36]+16*ind[46]+32*ind[53]+64*ind[58])*2.8125
    #s_2      = (1.0*ind[7]+2*ind[17]+4*ind[27]+8*ind[37]+16*ind[47])+1.0
    #omega_2  = (1.0*ind[8]+2*ind[18]+4*ind[28]+8*ind[38]+16*ind[48]+32*ind[54])*0.040+0.20
    #Hv_2     = (1.0*ind[9]+2.0*ind[19]+4.0*ind[29]+8.0*ind[39]+16.0*ind[49]+32.0*ind[55]+64.0*ind[59]+128.0*ind[61])*0.050
    for j in range(36):
        theta_func=j*10
        for i in range(25):
            omega_func=f_list[i]*2*math.pi
    
            if omega_func<=omega_1:
                s_1=math.ceil(s_max1*omega_func**5/omega_1**5)
            else:
                s_1=math.ceil(s_max1*omega_func**(-2.5)/omega_1**(-2.5))
            
            if omega_func<=omega_2:
                s_2=math.ceil(s_max2*omega_func**5/omega_2**5)
            else:
                s_2=math.ceil(s_max2*omega_func**(-2.5)/omega_2**(-2.5))
            
            spectrum_11=0.250*((4.0*lambda_1+1.0)/4.0*omega_1**4.0)**lambda_1
            spectrum_12=math.gamma(lambda_1)
            spectrum_16=Hv_1**2.0
            spectrum_13=omega_func**(4.0*lambda_1+1.0)
            spectrum_14=2.0**(2.0*s_1-1.0)*(math.gamma(s_1+1.0))**2.0/math.pi/math.gamma(2.0*s_1+1.0)
            spectrum_15=abs((math.cos((theta_func-theta_1)/2.0*math.pi/180.0)))**(2.0*s_1)*math.exp(-(4.0*lambda_1+1.0)/4.0*(omega_1/omega_func)**4.0)
            spectrum_1=spectrum_11/spectrum_13*spectrum_14*spectrum_15*spectrum_16/spectrum_12
            spectrum_21=0.250*((4.0*lambda_2+1.0)/4.0*omega_2**4.0)**lambda_2
            spectrum_22=math.gamma(lambda_2)
            spectrum_26=Hv_2**2.0
            spectrum_23=omega_func**(4.0*lambda_2+1.0)
            spectrum_24=2.0**(2.0*s_2-1.0)*(math.gamma(s_2+1.0))**2.0/math.pi/math.gamma(2.0*s_2+1.0)
            spectrum_25=abs((math.cos((theta_func-theta_2)/2.0*math.pi/180.0)))**(2.0*s_2)*math.exp(-(4.0*lambda_2+1.0)/4.0*(omega_2/omega_func)**4.0)
            spectrum_2=spectrum_21/spectrum_23*spectrum_24*spectrum_25*spectrum_26/spectrum_22
            if i==0:
                dw=f_list[1]-f_list[0]
            elif i==24:
                dw=f_list[24]-f_list[23]
            else:
                dw=(f_list[i+1]-f_list[i-1])/2
            errorfunc +=((wavedate.iloc[i][j+1]-2*math.pi*(spectrum_1+spectrum_2))*dw)**2.0
    return errorfunc


def adjustrating(C_p,M_p,CXPB_MAX,CXPB_MIN,M_MAX,M_MIN,f,f_max,fitness_mean):
    cross_para = 1.0/f-1/abs(f_max-fitness_mean)
    if (C_p*(1+cross_para))<=CXPB_MIN:
        C_p = CXPB_MIN
    else:
        if (C_p*(1+cross_para))<=CXPB_MAX:
            C_p = C_p*(1+cross_para)
        else:
            C_p = CXPB_MAX       
                                  
    M_para = 1/abs(f_max-fitness_mean) - 1/f
    if (M_p*(1+M_para))<=M_MIN:
        M_p = M_MIN
    else:
        if (M_p*(1+M_para))<=M_MAX:
            M_p = M_p*(1+M_para)
        else:
            M_p = M_MAX
         
    return  C_p,M_p

def randomselection(n_ind,pop,n_parents):
    chosen=[]    
    for i in range(n_parents):
        idx=random.randrange(0, n_ind-i)
        chosen.append(pop[idx])
        pop.pop(idx)
    return chosen,pop
    """chosen = []   
    eve1=random.randrange(0, n_ind, 2)
    odd1=random.randrange(1, n_ind, 2)
    chosen.append(pop[eve1])
    chosen.append(pop[odd1])
    if eve1>odd1:
        pop.pop(eve1)
        pop.pop(odd1)
    else:
        pop.pop(odd1)
        pop.pop(eve1) 
    return chosen,pop"""


def tournament(pop, n_ind, tournsize):
    """#Selection function."""
    chosen = []    
    for i in range(n_ind):
        aspirants = [random.choice(pop) for j in range(tournsize)]
        chosen.append(min(aspirants, key=attrgetter("fitness")))
    return chosen

def roulette(pop,C_max):
    """Selection function."""
    chosen=[]
    fitness_sum=0.0
    for i in range(len(pop)):
        fitness_sum += C_max- pop[i].fitness
        
    sigma_fitness = 0.0
    s = random.random()*fitness_sum
    for i in range(len(pop)):
        sigma_fitness += C_max-pop[i].fitness
        if (sigma_fitness)>=s:
            chosen.append(pop[i])
            break
    #chosens=chosen[:]
    #print("chosens"+str(len(chosens)))
    return chosen

def cxTwoPointCopy(ind1, ind2):
    """Crossover function."""
    size = len(ind1)
    tmp1 = ind1.copy()
    tmp2 = ind2.copy()
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size-1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1
    tmp1[cxpoint1:cxpoint2], tmp2[cxpoint1:cxpoint2] = tmp2[cxpoint1:cxpoint2].copy(), tmp1[cxpoint1:cxpoint2].copy()
    return tmp1, tmp2

def evalchild(offspring,n_ind):
    fit_mean = 0.0
    fit_var  = 0.0
    for i in range(n_ind):
        fit_mean += offspring[i].fitness/n_ind
    for k in range(n_ind):
        fit_var += (offspring[k].fitness - fit_mean)**2/n_ind
    return fit_mean, fit_var

def mutFlipBit(ind, indpb):
    """Mutation function."""
    tmp = ind.copy()
    for i in range(len(ind)):
        if random.random() < indpb:
            tmp[i] = type(ind[i])(not ind[i])
    return tmp

def keisan(n_ind,pop):
    Ex=0.0
    #Var=0.0
    for i in range(n_ind):
        Ex += pop[i].fitness
    #for i in range(n_ind):
        #Var += (pop[i].fitness-Ex)**2/n_ind
    return Ex#,Var
        
def get_parameter_from_ind(ind):
    lambda_1 = (1.0*ind[0]+2*ind[10]+4*ind[20]+8*ind[30]+16*ind[40])*0.10+0.10
    theta_1  = (1.0*ind[1]+2*ind[11]+4*ind[21]+8*ind[31]+16*ind[41]+32*ind[50]+64*ind[56])*2.8125
    s_1      = (1.0*ind[2]+2*ind[12]+4*ind[22]+8*ind[32]+16*ind[42]+32*ind[62])+1.0
    omega_1  = (1.0*ind[3]+2*ind[13]+4*ind[23]+8*ind[33]+16*ind[43]+32*ind[51])*0.040+0.20
    Hv_1     = (1.0*ind[4]+2*ind[14]+4*ind[24]+8*ind[34]+16*ind[44]+32.0*ind[52]+64.0*ind[57]+128.0*ind[60])*0.050
    lambda_2 = (1.0*ind[5]+2*ind[15]+4*ind[25]+8*ind[35]+16*ind[45])*0.10+0.10
    theta_2  = (1.0*ind[6]+2*ind[16]+4*ind[26]+8*ind[36]+16*ind[46]+32*ind[53]+64*ind[58])*2.8125
    s_2      = (1.0*ind[7]+2*ind[17]+4*ind[27]+8*ind[37]+16*ind[47]+32*ind[63])+1.0
    omega_2  = (1.0*ind[8]+2*ind[18]+4*ind[28]+8*ind[38]+16*ind[48]+32*ind[54])*0.040+0.20
    Hv_2     = (1.0*ind[9]+2.0*ind[19]+4.0*ind[29]+8.0*ind[39]+16.0*ind[49]+32.0*ind[55]+64.0*ind[59]+128.0*ind[61])*0.050# print("Generation loop ended. The best individual: ")
   #print("Hv1",Hv_1,"s1",s_1,"theta1",theta_1,"lambda1",lambda_1,"w1",omega_1)
    #print("Hv2",Hv_2,"s2",s_2,"theta2",theta_2,"lambda2",lambda_2,"w2",omega_2)
    # print(" fitness_mean "+str(fitness_mean)+" variance: " + str(fit_var))
    return lambda_1, s_1, theta_1, omega_1, Hv_1, lambda_2, s_2, theta_2, omega_2,Hv_2

if __name__ == "__main__":

    main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
